# 🚀 Smart Trading Journal - Model Training

**Notebook này giúp bạn train NLP & Behavioral models trên Google Colab với GPU miễn phí**

## Hướng dẫn sử dụng:
1. **Upload file này lên Google Colab**: Vào [colab.research.google.com](https://colab.research.google.com), chọn File > Upload notebook
2. **Bật GPU**: Runtime > Change runtime type > T4 GPU
3. **Chạy từng cell** hoặc Runtime > Run all

---

## 📦 Bước 1: Cài đặt dependencies

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install -q torch transformers datasets scikit-learn pandas numpy xgboost vaderSentiment langdetect

# Kiểm tra GPU
import torch
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

## 📂 Bước 2: Upload project files

Có 2 cách:
- **Cách 1**: Upload file zip (khuyến nghị)
- **Cách 2**: Mount Google Drive

In [ ]:
# Cách 1: Upload file zip
# Bạn cần zip thư mục backend và upload lên Colab

from google.colab import files
import zipfile
import os

print("📤 Upload file backend.zip từ máy tính của bạn...")
print("(Zip thư mục 'backend' trước khi upload)")
print()

uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/')
        print(f"✅ Extracted: {filename}")

In [ ]:
# Cách 2 (Alternative): Mount Google Drive
# Uncomment nếu bạn đã upload lên Drive

# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r /content/drive/MyDrive/smart-trading-journal/backend /content/

In [ ]:
# Thêm path vào sys.path
import sys
sys.path.insert(0, '/content/backend')

# Kiểm tra cấu trúc
!ls -la /content/backend/

## 🔧 Bước 3: Cấu hình Training

In [ ]:
# Cấu hình training
CONFIG = {
    # NLP Model
    "base_model": "ProsusAI/finbert",  # Hoặc "vinai/phobert-base" cho tiếng Việt
    "learning_rate": 2e-5,
    "batch_size": 16,
    "epochs": 3,
    "max_length": 128,
    
    # Labels
    "sentiment_labels": ["negative", "neutral", "positive"],
    "emotion_labels": ["FOMO", "FEAR", "GREED", "REVENGE", "RATIONAL", "CONFIDENT", "DISCIPLINE", "NEUTRAL"]
}

print("✅ Config loaded")
print(f"   Base model: {CONFIG['base_model']}")
print(f"   Epochs: {CONFIG['epochs']}")

## 📊 Bước 4: Chuẩn bị Dataset

In [ ]:
import json
import os

# Load dataset
dataset_path = "/content/backend/ml/training/nlp_dataset.json"

if os.path.exists(dataset_path):
    with open(dataset_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"✅ Dataset loaded: {len(data)} samples")
    print(f"\n📌 Sample data:")
    for i, sample in enumerate(data[:3]):
        print(f"   {i+1}. {sample['text'][:60]}... -> {sample.get('sentiment', 'N/A')}")
else:
    print("❌ Dataset not found! Creating sample dataset...")
    # Tạo sample dataset
    data = [
        {"text": "Phải vào ngay không thì lỡ cơ hội, giá đang tăng mạnh!", "sentiment": "negative", "emotion": "FOMO"},
        {"text": "Tôi đã phân tích kỹ chart và xác định entry point tốt", "sentiment": "positive", "emotion": "RATIONAL"},
        {"text": "Sợ quá, giá dump mạnh, phải cắt lỗ thôi", "sentiment": "negative", "emotion": "FEAR"},
        {"text": "Tiếp tục tuân thủ kế hoạch giao dịch đã đặt ra", "sentiment": "positive", "emotion": "DISCIPLINE"},
        {"text": "Thua rồi, phải gỡ lại ngay, all-in leverage x100", "sentiment": "negative", "emotion": "REVENGE"},
        {"text": "Chốt lời theo plan, tỷ lệ R:R như dự kiến", "sentiment": "positive", "emotion": "CONFIDENT"},
        {"text": "Thị trường đang sideway, nên chờ breakout", "sentiment": "neutral", "emotion": "RATIONAL"},
        {"text": "Coin này sẽ x10, all in không cần suy nghĩ!", "sentiment": "negative", "emotion": "GREED"},
    ]
    print(f"✅ Created sample dataset: {len(data)} samples")

## 🚀 Bước 5: Train NLP Model

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset

# Prepare data
texts = [d['text'] for d in data]
label_map = {label: i for i, label in enumerate(CONFIG['sentiment_labels'])}
labels = [label_map.get(d.get('sentiment', 'neutral'), 1) for d in data]

print(f"📊 Label distribution:")
for label, idx in label_map.items():
    count = labels.count(idx)
    print(f"   {label}: {count}")

# Split data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

print(f"\n✅ Train: {len(train_texts)}, Test: {len(test_texts)}")

In [ ]:
# Load tokenizer & model
print(f"📦 Loading model: {CONFIG['base_model']}")

tokenizer = AutoTokenizer.from_pretrained(CONFIG['base_model'])
model = AutoModelForSequenceClassification.from_pretrained(
    CONFIG['base_model'],
    num_labels=len(CONFIG['sentiment_labels'])
)

print("✅ Model loaded!")

In [ ]:
# Tokenize function
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=CONFIG['max_length']
    )

# Create datasets
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

print("✅ Data tokenized!")

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=CONFIG['learning_rate'],
    per_device_train_batch_size=CONFIG['batch_size'],
    per_device_eval_batch_size=CONFIG['batch_size'],
    num_train_epochs=CONFIG['epochs'],
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=10,
    report_to="none",
    fp16=True  # Sử dụng mixed precision cho GPU
)

# Compute metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(labels, predictions)}

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

print("✅ Trainer created!")

In [ ]:
# 🚀 TRAIN!
print("="*50)
print("🚀 Starting training...")
print("="*50)

trainer.train()

print("\n✅ Training complete!")

In [ ]:
# Evaluate
print("📊 Evaluating model...")

predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)

# Classification report
id_to_label = {v: k for k, v in label_map.items()}
print("\n" + classification_report(
    test_labels, preds,
    target_names=CONFIG['sentiment_labels']
))

## 💾 Bước 6: Lưu Model

In [ ]:
# Lưu model
output_dir = "./trained_model"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Model saved to {output_dir}")

# Zip để download
!zip -r trained_model.zip trained_model/
print("\n📦 Model zipped: trained_model.zip")

In [ ]:
# Download model về máy
from google.colab import files
files.download('trained_model.zip')

print("\n✅ Download complete!")
print("📁 Giải nén file và copy vào: backend/ml/models/finbert_trading_vi/")

## 🧪 Bước 7: Test Model (Optional)

In [ ]:
# Test với văn bản mới
test_texts = [
    "Phải mua ngay, giá đang tăng mạnh!",
    "Tôi đã phân tích kỹ và quyết định vào lệnh theo plan",
    "Sợ quá, bán hết đi thôi",
    "Hôm nay thị trường sideway, chờ đợi thêm"
]

print("🧪 Testing model predictions:\n")

for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred_idx = torch.argmax(probs).item()
        confidence = probs[0][pred_idx].item()
    
    label = CONFIG['sentiment_labels'][pred_idx]
    print(f"📝 \"{text[:40]}...\"")
    print(f"   → {label} (confidence: {confidence:.2%})\n")

---

## ✅ Hoàn tất!

Bạn đã train xong NLP model. Các bước tiếp theo:

1. Download file `trained_model.zip`
2. Giải nén và copy vào `backend/ml/models/finbert_trading_vi/`
3. Chạy lại backend để sử dụng model mới

---